In [ ]:
'''

以先前案例組合而成

S3 客戶端生成
DynamoDB 客戶端生成
flask app 生成

創建 S3 桶子
創建 DynamoDB Table

第一個方法
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳的表格清單，表格內要有 S3 的連結名

第二個方法
    用戶上傳圖片
    伺服器收到 Request
    生成 presigned url，上傳的檔案預設需公開
    並插入資料至 DynamoDB
    讓用戶將圖片傳至 presigned url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名


啟動flask app，使外部port 5000 能訪問


'''

In [ ]:
!pip install boto3 awscli flask

In [ ]:
'''

S3 客戶端生成
DynamoDB 客戶端生成
flask app 生成

'''

import boto3
from flask import Flask


s3_client = boto3.client('s3', endpoint_url='http://localstack-main:4566')

dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')

app = Flask(__name__)


In [ ]:
'''

創建 S3 桶子
創建 DynamoDB Table

'''

create_bucket_response = s3_client.create_bucket(
    Bucket='cxcxc-aws-certificate',
)
print(create_bucket_response)


table = dynamodb.create_table(
    TableName='s3-objects-list', 
    KeySchema=[
        {
            'AttributeName': 'object_name',
            'KeyType': 'HASH'  # Partition key
        }
    ], 
    AttributeDefinitions=[
        {
            'AttributeName': 'object_name',
            'AttributeType': 'S'  # N 代表數字； S 代表字串。
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print(table)

# 列出現有 bucket
response = s3_client.list_buckets()
print(response.get('Buckets'))

# 列出現有 table
table_list = list(dynamodb.tables.all())
print(table_list)

In [ ]:
'''

掃描 DynamoDB
將內含項目回傳

'''

def scan_dynamodb():
    dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')
    
    table = dynamodb.Table('s3-objects-list')

    # 掃描 DynamoDB 將所有 image 列出
    response = table.scan(
        FilterExpression=Attr('object_name').exists()
    )
    items = response['Items']
    print(items)
    return items

In [ ]:
'''

第一個方法
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳的表格清單，表格內要有 S3 的連結名

'''

from boto3.dynamodb.conditions import Key, Attr

@app.route('/view-object', methods=['GET'])
def view_object():
    
    # 掃描DynamoDB 項目
    items = scan_dynamodb()
    
    # 將項目顯示於 html 頁面
    html_object = ""
    for item in items:
        object_name = item.get('object_name')
        object_url = "http://localstack-main:4566/cxcxc-aws-certificate/" + object_name
        
        html_object += f"""
        <tr>
            <td>{object_name}</td>
            <td>{object_url}</td>
        </tr>
        """
    
    # 準備 html 頁面
    html = f"""<!doctype html>
        <title>Read Files</title>
        <h1>Read Files</h1>

        <table border="1">
            <tr>
                <td>物件名稱</td>
                <td>URL</td>
            </tr>
            {html_object}
        </table>
    """
    
    return html

In [ ]:
'''

根據時間為物件取名
生成S3 presigned post

'''

import datetime

def create_presigned_post(bucket_name='cxcxc-aws-certificate', object_name=None) -> dict:
    
    s3_client = boto3.client('s3', endpoint_url='http://localstack-main:4566')

    if object_name is None:
        object_name = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    
    # Generate a presigned S3 POST URL
    response = s3_client.generate_presigned_post(bucket_name, object_name)
    
    # The response contains the presigned URL and required fields
    return response

In [ ]:
'''

將S3 物件名稱存入DynamoDB

'''

def put_item_to_dynamodb(s3_key, table_name='s3-objects-list'):
    
    dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')
    table = dynamodb.Table(table_name)
    
    table.put_item(
       Item={
            "object_name": s3_key
        }
    )
    

In [ ]:
'''

第二個方法
    用戶上傳圖片
    伺服器收到 Request
    生成 presigned url，上傳的檔案預設需公開
    並插入資料至 DynamoDB
    讓用戶將圖片傳至 presigned url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

'''

@app.route('/upload-object', methods=['GET', 'POST'])
def upload_object():
    
    # 生成S3 presigned post
    presigned_post = create_presigned_post()
    print(presigned_post)
    presigned_post_url = presigned_post.get('url')
    required_fields = presigned_post.get('fields')
    
    # 將S3 key 存入DynamoDB
    object_key = required_fields.get('key')
    put_item_to_dynamodb(s3_key=object_key)
    
    # 掃描DynamoDB 項目
    items = scan_dynamodb()
    
    # 將項目顯示於 html 頁面
    html_object = ""
    for item in items:
        object_name = item.get('object_name')
        object_url = "http://localstack-main:4566/cxcxc-aws-certificate/" + object_name
        
        html_object += f"""
        <tr>
            <td>{object_name}</td>
            <td>{object_url}</td>
        </tr>
        """
    
    # 準備 html 頁面
    html = f"""<html>
      <head>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
      </head>
      <body>
        <form action="{presigned_post_url}" method="post" enctype="multipart/form-data"> 
          <input type="hidden" name="key" value="{required_fields.get('key')}" />
          <input type="hidden" name="AWSAccessKeyId" value="{required_fields.get('AWSAccessKeyId')}" />
          <input type="hidden" name="policy" value="{required_fields.get('policy')}" />
          <input type="hidden" name="signature" value="{required_fields.get('signature')}" />
        File:
          <input type="file"   name="file" /> <br />
          <input type="submit" name="submit" value="Upload to Amazon S3"  />
        </form>
        
        <table border="1">
            <tr>
                <td>物件名稱</td>
                <td>URL</td>
            </tr>
            {html_object}
        </table>
        
      </body>
    </html>
    """
    
    return html



In [ ]:
'''

啟動flask app，使外部port 5000 能訪問

'''

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)